In [1]:
%load_ext autoreload
%autoreload 2
import pinot
import torch
import numpy as np
from itertools import chain


Using backend: pytorch


In [2]:
ds = pinot.data.moonshot_meta()

In [3]:
ds_tr, ds_te = pinot.data.utils.split(ds, [4, 1])

In [59]:
def get_separate_dataset(ds):
    # n_tasks = ds[0][1].shape[-1]
    n_tasks = 6
    datasets = [[] for _ in range(n_tasks)]
    
    for g, y in ds:
        for idx in range(n_tasks):
            if np.isnan(y[idx].numpy()) == False:
                datasets[idx].append((g, y[idx][None]))
    
    return datasets


In [60]:
datasets = get_separate_dataset(ds_tr)
datasets = [pinot.data.utils.batch(data, len(data)) for data in datasets]

datasets_te = get_separate_dataset(ds_te)
datasets_te = [pinot.data.utils.batch(data, len(data)) for data in datasets_te]

In [82]:
net = pinot.representation.Sequential(
    pinot.representation.dgl_legacy.gn(),
    [32, 'tanh', 32, 'tanh', 32, 'tanh'])

In [83]:
nets = []
for idx in range(len(datasets)):
    
    model = pinot.Net(
        net)
    
    nets.append(model)

In [84]:
params = []

for net in nets:
    params += list(net.parameters())

opt = pinot.BBB(
        torch.optim.Adam(params, 1e-5),
        initializer_std=0.001)
for _ in range(500):
    opt.zero_grad()
    
    def l():
        loss = 0.0
        for idx, net in enumerate(nets):
            g, y = datasets[idx][0]
            loss += net.loss(g, y).mean()
        print(loss)
        loss.backward()
        return loss
    opt.step(l)
        

tensor(1.4119e+09, grad_fn=<AddBackward0>)
tensor(1.3039e+09, grad_fn=<AddBackward0>)
tensor(6.9860e+08, grad_fn=<AddBackward0>)
tensor(3.4749e+08, grad_fn=<AddBackward0>)
tensor(75916720., grad_fn=<AddBackward0>)
tensor(94143112., grad_fn=<AddBackward0>)
tensor(72633144., grad_fn=<AddBackward0>)
tensor(32551780., grad_fn=<AddBackward0>)
tensor(8581598., grad_fn=<AddBackward0>)
tensor(3789874., grad_fn=<AddBackward0>)
tensor(6363516.5000, grad_fn=<AddBackward0>)
tensor(574141.3125, grad_fn=<AddBackward0>)
tensor(935707.1250, grad_fn=<AddBackward0>)
tensor(921985.9375, grad_fn=<AddBackward0>)
tensor(624274., grad_fn=<AddBackward0>)
tensor(225498.9375, grad_fn=<AddBackward0>)
tensor(267486.1875, grad_fn=<AddBackward0>)
tensor(159742.9844, grad_fn=<AddBackward0>)
tensor(20128.7578, grad_fn=<AddBackward0>)
tensor(84336.8672, grad_fn=<AddBackward0>)
tensor(46082.6055, grad_fn=<AddBackward0>)
tensor(47264.3477, grad_fn=<AddBackward0>)
tensor(59534.1484, grad_fn=<AddBackward0>)
tensor(69333.9

tensor(928049.1250, grad_fn=<AddBackward0>)
tensor(636814.3750, grad_fn=<AddBackward0>)
tensor(683185., grad_fn=<AddBackward0>)
tensor(343398.8438, grad_fn=<AddBackward0>)
tensor(56072.3750, grad_fn=<AddBackward0>)
tensor(47126.2891, grad_fn=<AddBackward0>)
tensor(72742.0859, grad_fn=<AddBackward0>)
tensor(84149.7344, grad_fn=<AddBackward0>)
tensor(56133.9062, grad_fn=<AddBackward0>)
tensor(235280.9688, grad_fn=<AddBackward0>)
tensor(176464.6250, grad_fn=<AddBackward0>)
tensor(930218.7500, grad_fn=<AddBackward0>)
tensor(410380.6562, grad_fn=<AddBackward0>)
tensor(1559170.1250, grad_fn=<AddBackward0>)
tensor(1016399.1250, grad_fn=<AddBackward0>)
tensor(464451.0312, grad_fn=<AddBackward0>)
tensor(1010852.0625, grad_fn=<AddBackward0>)
tensor(2732095.5000, grad_fn=<AddBackward0>)
tensor(3238218.2500, grad_fn=<AddBackward0>)
tensor(1311582.5000, grad_fn=<AddBackward0>)
tensor(1288578.5000, grad_fn=<AddBackward0>)
tensor(680300.1250, grad_fn=<AddBackward0>)
tensor(710595.3125, grad_fn=<AddBa

tensor(2476435.7500, grad_fn=<AddBackward0>)
tensor(325054.9062, grad_fn=<AddBackward0>)
tensor(561214.7500, grad_fn=<AddBackward0>)
tensor(391391.4688, grad_fn=<AddBackward0>)
tensor(1046286.5000, grad_fn=<AddBackward0>)
tensor(3411432.2500, grad_fn=<AddBackward0>)
tensor(946660.5000, grad_fn=<AddBackward0>)
tensor(2474349.5000, grad_fn=<AddBackward0>)
tensor(1394052.5000, grad_fn=<AddBackward0>)
tensor(866843.7500, grad_fn=<AddBackward0>)
tensor(3465792., grad_fn=<AddBackward0>)
tensor(866898.6875, grad_fn=<AddBackward0>)
tensor(424561.7500, grad_fn=<AddBackward0>)
tensor(57380.9688, grad_fn=<AddBackward0>)
tensor(357814.2188, grad_fn=<AddBackward0>)
tensor(799957.7500, grad_fn=<AddBackward0>)
tensor(1566216.8750, grad_fn=<AddBackward0>)
tensor(4322315., grad_fn=<AddBackward0>)
tensor(1566516.7500, grad_fn=<AddBackward0>)
tensor(2919183.5000, grad_fn=<AddBackward0>)
tensor(474075.8125, grad_fn=<AddBackward0>)
tensor(107101.9375, grad_fn=<AddBackward0>)
tensor(55697.9062, grad_fn=<Add

In [85]:
res = []


metrics = ['avg_nll', 'r2']
for idx, net in enumerate(nets):
    g, y = datasets_te[idx][0]
    
    print('-----')
    
    print(idx)
    for metric in metrics:
        _metric = getattr(pinot.metrics, metric)(net, g, y).detach().numpy()
        res.append('%.2f' % _metric)
        


-----
0
-----
1
-----
2
-----
3
-----
4
-----
5


In [86]:
print('&'.join(res))

19.92&-126.11&26614.86&-269.10&1115.19&-144.68&6.46&-487.88&1.40&-4.54&45.99&-1048.60


In [80]:
res = []

nets = []

for idx in range(len(datasets)):

    net = pinot.representation.Sequential(
        pinot.representation.dgl_legacy.gn(),
        [32, 'tanh', 32, 'tanh', 32, 'tanh'])


    model = pinot.Net(
        net)

    nets.append(model)

    
for idx, net in enumerate(nets):
    opt = pinot.BBB(torch.optim.Adam(net.parameters(), 1e-5), initializer_std=0.001)
    
    for _ in range(500):

        opt.zero_grad()

        def l():
            g, y = datasets[idx][0]
            loss = net.loss(g, y).mean()
            print(loss)
            loss.backward()
            return loss
        
        opt.step(l)
        
    print('-----')

    print(idx)
    for metric in metrics:
        g, y = datasets_te[idx][0]
        _metric = getattr(pinot.metrics, metric)(net, g, y).detach().numpy()
        res.append('%.2f' % _metric)
        

tensor(4.8332, grad_fn=<MeanBackward0>)
tensor(4.5616, grad_fn=<MeanBackward0>)
tensor(4.6158, grad_fn=<MeanBackward0>)
tensor(4.3885, grad_fn=<MeanBackward0>)
tensor(4.4010, grad_fn=<MeanBackward0>)
tensor(4.5593, grad_fn=<MeanBackward0>)
tensor(4.4834, grad_fn=<MeanBackward0>)
tensor(4.3543, grad_fn=<MeanBackward0>)
tensor(4.1714, grad_fn=<MeanBackward0>)
tensor(4.0292, grad_fn=<MeanBackward0>)
tensor(4.0401, grad_fn=<MeanBackward0>)
tensor(4.1740, grad_fn=<MeanBackward0>)
tensor(4.2407, grad_fn=<MeanBackward0>)
tensor(4.1159, grad_fn=<MeanBackward0>)
tensor(4.1274, grad_fn=<MeanBackward0>)
tensor(4.0517, grad_fn=<MeanBackward0>)
tensor(4.0587, grad_fn=<MeanBackward0>)
tensor(3.9837, grad_fn=<MeanBackward0>)
tensor(3.9557, grad_fn=<MeanBackward0>)
tensor(4.0795, grad_fn=<MeanBackward0>)
tensor(3.8612, grad_fn=<MeanBackward0>)
tensor(3.9081, grad_fn=<MeanBackward0>)
tensor(3.9223, grad_fn=<MeanBackward0>)
tensor(3.8237, grad_fn=<MeanBackward0>)
tensor(3.7612, grad_fn=<MeanBackward0>)


tensor(0.4952, grad_fn=<MeanBackward0>)
tensor(0.3313, grad_fn=<MeanBackward0>)
tensor(0.2941, grad_fn=<MeanBackward0>)
tensor(0.4467, grad_fn=<MeanBackward0>)
tensor(0.3667, grad_fn=<MeanBackward0>)
tensor(0.3584, grad_fn=<MeanBackward0>)
tensor(0.3937, grad_fn=<MeanBackward0>)
tensor(0.3948, grad_fn=<MeanBackward0>)
tensor(0.6102, grad_fn=<MeanBackward0>)
tensor(0.5021, grad_fn=<MeanBackward0>)
tensor(0.3180, grad_fn=<MeanBackward0>)
tensor(0.3672, grad_fn=<MeanBackward0>)
tensor(0.3817, grad_fn=<MeanBackward0>)
tensor(0.3669, grad_fn=<MeanBackward0>)
tensor(0.3979, grad_fn=<MeanBackward0>)
tensor(0.3515, grad_fn=<MeanBackward0>)
tensor(0.3811, grad_fn=<MeanBackward0>)
tensor(0.4390, grad_fn=<MeanBackward0>)
tensor(0.3663, grad_fn=<MeanBackward0>)
tensor(0.3918, grad_fn=<MeanBackward0>)
tensor(0.3341, grad_fn=<MeanBackward0>)
tensor(0.3214, grad_fn=<MeanBackward0>)
tensor(0.5771, grad_fn=<MeanBackward0>)
tensor(0.5533, grad_fn=<MeanBackward0>)
tensor(0.8655, grad_fn=<MeanBackward0>)


tensor(0.3030, grad_fn=<MeanBackward0>)
tensor(0.2446, grad_fn=<MeanBackward0>)
tensor(0.2976, grad_fn=<MeanBackward0>)
tensor(0.4571, grad_fn=<MeanBackward0>)
tensor(0.5975, grad_fn=<MeanBackward0>)
tensor(0.3994, grad_fn=<MeanBackward0>)
tensor(0.3876, grad_fn=<MeanBackward0>)
tensor(0.7433, grad_fn=<MeanBackward0>)
tensor(0.2468, grad_fn=<MeanBackward0>)
tensor(0.3895, grad_fn=<MeanBackward0>)
tensor(0.3309, grad_fn=<MeanBackward0>)
tensor(0.8664, grad_fn=<MeanBackward0>)
tensor(0.3536, grad_fn=<MeanBackward0>)
tensor(0.2538, grad_fn=<MeanBackward0>)
tensor(0.3739, grad_fn=<MeanBackward0>)
tensor(0.3976, grad_fn=<MeanBackward0>)
tensor(0.3214, grad_fn=<MeanBackward0>)
tensor(0.3975, grad_fn=<MeanBackward0>)
tensor(0.3719, grad_fn=<MeanBackward0>)
tensor(0.2561, grad_fn=<MeanBackward0>)
tensor(0.2281, grad_fn=<MeanBackward0>)
tensor(0.2855, grad_fn=<MeanBackward0>)
tensor(0.8179, grad_fn=<MeanBackward0>)
tensor(0.3605, grad_fn=<MeanBackward0>)
tensor(0.2449, grad_fn=<MeanBackward0>)


tensor(3.8326, grad_fn=<MeanBackward0>)
tensor(3.8119, grad_fn=<MeanBackward0>)
tensor(3.8217, grad_fn=<MeanBackward0>)
tensor(3.8238, grad_fn=<MeanBackward0>)
tensor(3.8263, grad_fn=<MeanBackward0>)
tensor(3.8589, grad_fn=<MeanBackward0>)
tensor(3.8011, grad_fn=<MeanBackward0>)
tensor(3.9067, grad_fn=<MeanBackward0>)
tensor(3.8143, grad_fn=<MeanBackward0>)
tensor(3.8077, grad_fn=<MeanBackward0>)
tensor(3.8017, grad_fn=<MeanBackward0>)
tensor(3.8577, grad_fn=<MeanBackward0>)
tensor(3.8120, grad_fn=<MeanBackward0>)
tensor(3.8012, grad_fn=<MeanBackward0>)
tensor(3.8293, grad_fn=<MeanBackward0>)
tensor(3.8052, grad_fn=<MeanBackward0>)
tensor(3.8122, grad_fn=<MeanBackward0>)
tensor(3.7921, grad_fn=<MeanBackward0>)
tensor(3.8126, grad_fn=<MeanBackward0>)
tensor(3.8825, grad_fn=<MeanBackward0>)
tensor(3.7918, grad_fn=<MeanBackward0>)
tensor(3.8102, grad_fn=<MeanBackward0>)
tensor(3.8108, grad_fn=<MeanBackward0>)
tensor(3.8543, grad_fn=<MeanBackward0>)
tensor(3.8108, grad_fn=<MeanBackward0>)


tensor(3.7594, grad_fn=<MeanBackward0>)
tensor(3.7675, grad_fn=<MeanBackward0>)
tensor(3.7535, grad_fn=<MeanBackward0>)
tensor(3.8430, grad_fn=<MeanBackward0>)
tensor(3.7619, grad_fn=<MeanBackward0>)
tensor(3.7375, grad_fn=<MeanBackward0>)
tensor(3.7893, grad_fn=<MeanBackward0>)
tensor(3.7948, grad_fn=<MeanBackward0>)
tensor(3.7652, grad_fn=<MeanBackward0>)
tensor(3.7645, grad_fn=<MeanBackward0>)
tensor(3.8613, grad_fn=<MeanBackward0>)
tensor(3.7686, grad_fn=<MeanBackward0>)
tensor(3.7607, grad_fn=<MeanBackward0>)
tensor(3.7565, grad_fn=<MeanBackward0>)
tensor(3.7656, grad_fn=<MeanBackward0>)
tensor(3.7500, grad_fn=<MeanBackward0>)
tensor(3.7465, grad_fn=<MeanBackward0>)
tensor(3.7551, grad_fn=<MeanBackward0>)
tensor(3.7555, grad_fn=<MeanBackward0>)
tensor(3.7898, grad_fn=<MeanBackward0>)
tensor(3.7406, grad_fn=<MeanBackward0>)
tensor(3.7954, grad_fn=<MeanBackward0>)
tensor(3.7796, grad_fn=<MeanBackward0>)
tensor(3.7594, grad_fn=<MeanBackward0>)
tensor(3.7390, grad_fn=<MeanBackward0>)


...

tensor(4.1642, grad_fn=<MeanBackward0>)
tensor(4.6216, grad_fn=<MeanBackward0>)
tensor(4.0189, grad_fn=<MeanBackward0>)
tensor(3.8840, grad_fn=<MeanBackward0>)
tensor(4.2281, grad_fn=<MeanBackward0>)
tensor(3.5062, grad_fn=<MeanBackward0>)
tensor(3.7508, grad_fn=<MeanBackward0>)
tensor(3.6309, grad_fn=<MeanBackward0>)
tensor(3.6406, grad_fn=<MeanBackward0>)
tensor(3.6923, grad_fn=<MeanBackward0>)
tensor(3.3324, grad_fn=<MeanBackward0>)
tensor(3.4515, grad_fn=<MeanBackward0>)
tensor(3.5998, grad_fn=<MeanBackward0>)
tensor(3.1768, grad_fn=<MeanBackward0>)
tensor(3.4773, grad_fn=<MeanBackward0>)
tensor(3.2509, grad_fn=<MeanBackward0>)
tensor(2.7606, grad_fn=<MeanBackward0>)
tensor(3.2905, grad_fn=<MeanBackward0>)
tensor(2.9224, grad_fn=<MeanBackward0>)
tensor(2.9933, grad_fn=<MeanBackward0>)
tensor(2.7933, grad_fn=<MeanBackward0>)
tensor(2.7435, grad_fn=<MeanBackward0>)
tensor(2.5498, grad_fn=<MeanBackward0>)
tensor(2.6200, grad_fn=<MeanBackward0>)
tensor(2.1136, grad_fn=<MeanBackward0>)


tensor(0.5977, grad_fn=<MeanBackward0>)
tensor(0.6051, grad_fn=<MeanBackward0>)
tensor(0.6932, grad_fn=<MeanBackward0>)
tensor(0.4444, grad_fn=<MeanBackward0>)
tensor(0.4311, grad_fn=<MeanBackward0>)
tensor(0.7780, grad_fn=<MeanBackward0>)
tensor(0.5753, grad_fn=<MeanBackward0>)
tensor(0.6288, grad_fn=<MeanBackward0>)
tensor(0.4594, grad_fn=<MeanBackward0>)
tensor(0.4700, grad_fn=<MeanBackward0>)
tensor(0.6128, grad_fn=<MeanBackward0>)
tensor(0.5288, grad_fn=<MeanBackward0>)
tensor(0.4850, grad_fn=<MeanBackward0>)
tensor(0.5169, grad_fn=<MeanBackward0>)
tensor(0.6605, grad_fn=<MeanBackward0>)
tensor(0.4976, grad_fn=<MeanBackward0>)
tensor(0.4955, grad_fn=<MeanBackward0>)
tensor(0.4535, grad_fn=<MeanBackward0>)
tensor(0.4545, grad_fn=<MeanBackward0>)
tensor(0.5703, grad_fn=<MeanBackward0>)
tensor(0.6181, grad_fn=<MeanBackward0>)
tensor(0.4370, grad_fn=<MeanBackward0>)
tensor(0.5152, grad_fn=<MeanBackward0>)
tensor(1.2325, grad_fn=<MeanBackward0>)
tensor(0.4473, grad_fn=<MeanBackward0>)


tensor(0.6156, grad_fn=<MeanBackward0>)
tensor(0.7938, grad_fn=<MeanBackward0>)
tensor(0.4628, grad_fn=<MeanBackward0>)
tensor(0.4225, grad_fn=<MeanBackward0>)
tensor(0.5358, grad_fn=<MeanBackward0>)
tensor(0.4504, grad_fn=<MeanBackward0>)
tensor(0.4336, grad_fn=<MeanBackward0>)
tensor(0.4298, grad_fn=<MeanBackward0>)
tensor(0.4624, grad_fn=<MeanBackward0>)
tensor(0.4945, grad_fn=<MeanBackward0>)
tensor(0.4672, grad_fn=<MeanBackward0>)
tensor(0.4261, grad_fn=<MeanBackward0>)
tensor(0.4587, grad_fn=<MeanBackward0>)
tensor(0.5730, grad_fn=<MeanBackward0>)
tensor(0.5671, grad_fn=<MeanBackward0>)
tensor(0.4016, grad_fn=<MeanBackward0>)
tensor(0.6879, grad_fn=<MeanBackward0>)
tensor(0.7167, grad_fn=<MeanBackward0>)
tensor(0.6188, grad_fn=<MeanBackward0>)
tensor(0.4337, grad_fn=<MeanBackward0>)
tensor(0.6508, grad_fn=<MeanBackward0>)
tensor(0.3811, grad_fn=<MeanBackward0>)
tensor(0.4374, grad_fn=<MeanBackward0>)
tensor(0.4430, grad_fn=<MeanBackward0>)
tensor(0.9596, grad_fn=<MeanBackward0>)


tensor(4.1403, grad_fn=<MeanBackward0>)
tensor(3.6396, grad_fn=<MeanBackward0>)
tensor(4.0769, grad_fn=<MeanBackward0>)
tensor(3.7591, grad_fn=<MeanBackward0>)
tensor(3.7535, grad_fn=<MeanBackward0>)
tensor(4.4004, grad_fn=<MeanBackward0>)
tensor(3.9219, grad_fn=<MeanBackward0>)
tensor(3.3630, grad_fn=<MeanBackward0>)
tensor(3.4112, grad_fn=<MeanBackward0>)
tensor(3.5898, grad_fn=<MeanBackward0>)
tensor(4.5581, grad_fn=<MeanBackward0>)
tensor(4.0263, grad_fn=<MeanBackward0>)
tensor(3.9087, grad_fn=<MeanBackward0>)
tensor(3.3483, grad_fn=<MeanBackward0>)
tensor(3.7776, grad_fn=<MeanBackward0>)
tensor(5.0201, grad_fn=<MeanBackward0>)
tensor(3.1455, grad_fn=<MeanBackward0>)
tensor(4.6361, grad_fn=<MeanBackward0>)
tensor(3.1767, grad_fn=<MeanBackward0>)
tensor(3.8340, grad_fn=<MeanBackward0>)
tensor(3.1644, grad_fn=<MeanBackward0>)
tensor(4.3692, grad_fn=<MeanBackward0>)
tensor(3.3247, grad_fn=<MeanBackward0>)
tensor(3.5453, grad_fn=<MeanBackward0>)
tensor(3.0265, grad_fn=<MeanBackward0>)


tensor(2.7349, grad_fn=<MeanBackward0>)
tensor(2.7959, grad_fn=<MeanBackward0>)
tensor(2.7230, grad_fn=<MeanBackward0>)
tensor(2.7555, grad_fn=<MeanBackward0>)
tensor(2.8240, grad_fn=<MeanBackward0>)
tensor(2.6980, grad_fn=<MeanBackward0>)
tensor(2.7455, grad_fn=<MeanBackward0>)
tensor(2.6839, grad_fn=<MeanBackward0>)
tensor(2.6021, grad_fn=<MeanBackward0>)
tensor(2.9206, grad_fn=<MeanBackward0>)
tensor(2.8274, grad_fn=<MeanBackward0>)
tensor(3.0611, grad_fn=<MeanBackward0>)
tensor(2.7835, grad_fn=<MeanBackward0>)
tensor(3.2063, grad_fn=<MeanBackward0>)
tensor(2.7090, grad_fn=<MeanBackward0>)
tensor(3.1444, grad_fn=<MeanBackward0>)
tensor(2.8368, grad_fn=<MeanBackward0>)
tensor(2.7149, grad_fn=<MeanBackward0>)
tensor(2.6503, grad_fn=<MeanBackward0>)
tensor(2.8701, grad_fn=<MeanBackward0>)
tensor(2.6918, grad_fn=<MeanBackward0>)
tensor(2.7952, grad_fn=<MeanBackward0>)
tensor(2.8769, grad_fn=<MeanBackward0>)
tensor(3.2824, grad_fn=<MeanBackward0>)
tensor(2.7273, grad_fn=<MeanBackward0>)


In [81]:
print('&'.join(res))

0.18&-1.01&3.72&-803.79&0.37&-2.74&37.72&-46.31&0.46&-0.30&2.61&-159.65
